leave-one-nuclide-out loop - predicting all nuclides 10 times and savind data to csv file

Best hyperparameters found for each lambda are used here. Run the cells below 

In [ ]:
pip install pandas numpy scikit-learn torch optuna matplotlib

In [ ]:


# --------------------------- IMPORTS ----------------------------------
import random, pathlib, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --------------------------- CONSTANTS --------------------------------
CSV_PATH   = "merged_8_july_d_lowmassremoved.csv"          # master dataset
BASE_OUT   = pathlib.Path("results_lam0")   # parent folder for all isotopes
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

# ---- fixed hyper-parameters (λ = 0 run) ----
LAM        = 0.0
N_HIDDEN   = 96
N_LAYERS   = 2
DROPOUT    = 0.23455646448047404
LR         = 0.001259422015017642

MAX_EPOCHS = 200
PATIENCE   = 15
BATCH_SHUFFLE = True

N_RUNS     = 10
SEED_BASE  = 1234
EPS        = 1e-12

FEATURES = [
    'ERG','Radius','n_rms_radius','p_rms_radius','octopole_deformation',
    'n_chem_erg','Sn','Z','S2p_compound','gamma_deformation','ME',
    'BEA_A_daughter','A','Radius_daughter','BEA_A','AM','Radius_compound',
    'Pairing_daughter','BEA_compound','beta_deformation','Theory_thresh',
    'shell_P','pairing_delta','symmetry_S','excitation_energy',
    'Thresh_n3n','Excite_n3n','Thresh_n4n','Excite_n4n'
]
TARGET, SIGMA_EXP, FLUENCE, GAMMA, ISO = (
    "XS","sigma_exp","fluence","gamma","iso_id"
)

# --------------------------- UTILITIES --------------------------------
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ----------------------- DATASET & SAMPLER ----------------------------
class XSData(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler,
                 fit_scaler: bool = False):
        if fit_scaler:
            scaler.fit(df[FEATURES])
        self.X = torch.tensor(scaler.transform(df[FEATURES]).astype(np.float32))
        self.y = torch.tensor(df[TARGET].to_numpy(np.float32))
        self.fluence   = torch.tensor(df[FLUENCE].to_numpy(np.float32))
        self.sigma_exp = torch.tensor(df[SIGMA_EXP].fillna(0).to_numpy(np.float32))
        self.gamma     = torch.tensor(df[GAMMA].to_numpy(np.float32))
        self.iso_code  = df[ISO].astype("category").cat.codes.to_numpy()

    def __len__(self): return len(self.y)

    def __getitem__(self, idx: int):
        return {"x": self.X[idx], "y": self.y[idx],
                "fluence": self.fluence[idx],
                "sigma_exp": self.sigma_exp[idx],
                "gamma": self.gamma[idx],
                "iso_code": self.iso_code[idx]}

class IsoBatchSampler(Sampler):
    def __init__(self, iso_codes, shuffle=True):
        self.indices = {}
        for i, iso in enumerate(iso_codes):
            self.indices.setdefault(int(iso), []).append(i)
        self.keys = list(self.indices.keys())
        self.shuffle = shuffle

    def __iter__(self):
        keys = self.keys.copy()
        if self.shuffle: random.shuffle(keys)
        for k in keys:
            yield self.indices[k]

    def __len__(self): return len(self.keys)

# ------------------------------ MODEL ---------------------------------
class MLP(nn.Module):
    def __init__(self, n_in:int, n_hidden:int, n_layers:int, p_drop:float):
        super().__init__()
        layers = []
        for l in range(n_layers):
            layers += [nn.Linear(n_in if l==0 else n_hidden, n_hidden),
                       nn.ReLU(), nn.Dropout(p_drop)]
        layers.append(nn.Linear(n_hidden, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x).squeeze(1)

# -------------------------- CUSTOM LOSS -------------------------------
def iso_loss(batch, preds, lam:float):
    mse = nn.functional.mse_loss(preds, batch["y"])
    if batch["gamma"][0] == 0:     # skip σ₁g term if γ = 0
        return mse
    flu        = batch["fluence"]
    sigma_pred = (flu * preds).sum() / (flu.sum() + EPS)
    sigma_true = batch["sigma_exp"][0]
    rel_err_sq = ((sigma_pred - sigma_true) / (sigma_true + EPS))**2
    return mse + lam * rel_err_sq

# ------------------------ TRAIN / VALID LOOP --------------------------
def run_epoch(model, loader, opt, lam):
    is_train = opt is not None
    model.train(is_train)
    total, n = 0.0, 0
    for batch in loader:
        batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                 for k,v in batch.items()}
        preds = model(batch["x"])
        loss  = iso_loss(batch, preds, lam)
        if is_train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        total += loss.item() * len(preds)
        n     += len(preds)
    return total / n

# ----------------------- PER-ISOTOPE ROUTINE --------------------------
def process_isotope(df_all: pd.DataFrame, iso_hold: str) -> None:
    out_dir = BASE_OUT / iso_hold
    out_dir.mkdir(parents=True, exist_ok=True)

    df_test  = df_all[df_all[ISO] == iso_hold].reset_index(drop=True)
    df_train = df_all[df_all[ISO] != iso_hold].reset_index(drop=True)
    if df_test.empty:
        print(f"⚠  No rows for iso_id={iso_hold}; skipping.")
        return

    preds_runs = []
    for run in range(N_RUNS):
        set_seed(SEED_BASE + run)

        scaler = StandardScaler()
        ds_tr  = XSData(df_train, scaler, fit_scaler=True)
        ds_te  = XSData(df_test,  scaler, fit_scaler=False)

        dl_tr = DataLoader(ds_tr, batch_sampler=IsoBatchSampler(ds_tr.iso_code,
                                                                shuffle=BATCH_SHUFFLE))
        dl_te = DataLoader(ds_te, batch_sampler=IsoBatchSampler(ds_te.iso_code,
                                                                shuffle=False))

        model = MLP(len(FEATURES), N_HIDDEN, N_LAYERS, DROPOUT).to(DEVICE)
        opt   = torch.optim.Adam(model.parameters(), lr=LR)

        best, patience = float("inf"), 0
        for _ in range(MAX_EPOCHS):
            loss_tr = run_epoch(model, dl_tr, opt, LAM)
            if loss_tr < best - 1e-6:
                best, patience = loss_tr, 0
            else:
                patience += 1
                if patience >= PATIENCE:
                    break

        # predict
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in dl_te:
                batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                         for k,v in batch.items()}
                preds.append(model(batch["x"]).cpu())
        preds_runs.append(torch.cat(preds).numpy())

    preds_arr  = np.stack(preds_runs, axis=1)
    pred_mean  = preds_arr.mean(axis=1)

    # save CSV ----------------------------------------------------------
    df_out = pd.DataFrame({"ERG": df_test["ERG"], "XS_true": df_test["XS"]})
    for r in range(N_RUNS):
        df_out[f"pred_run{r+1}"] = preds_arr[:, r]
    df_out["pred_mean"] = pred_mean
    csv_path = out_dir / f"{iso_hold}_predictions.csv"
    df_out.to_csv(csv_path, index=False)

    # plot --------------------------------------------------------------
    plt.figure(figsize=(6,4))
    plt.plot(df_out["ERG"], df_out["XS_true"],  lw=2, label="True XS")
    plt.plot(df_out["ERG"], df_out["pred_mean"], lw=2, ls="--",
             label=f"Mean pred ({N_RUNS}×)")
    plt.xlabel("ERG"); plt.ylabel("XS")
    plt.title(f"{iso_hold} – true vs predicted XS")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / f"{iso_hold}_erg_vs_pred.png", dpi=300)
    plt.close()

    # meta --------------------------------------------------------------
    meta = dict(lam=LAM, n_hidden=N_HIDDEN, n_layers=N_LAYERS,
                dropout=DROPOUT, lr=LR, n_runs=N_RUNS,
                iso_held_out=iso_hold)
    with open(out_dir / "run_meta.json", "w") as fh:
        json.dump(meta, fh, indent=2)

    print(f"✓ finished {iso_hold}")

# ------------------------------- MAIN ---------------------------------
def main() -> None:
    BASE_OUT.mkdir(parents=True, exist_ok=True)
    df_all   = pd.read_csv(CSV_PATH)
    iso_list = sorted(df_all[ISO].unique())

    print(f"Found {len(iso_list)} unique isotopes.")
    for iso in iso_list:
        process_isotope(df_all, iso)

    print("\nAll isotopes done!")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()


In [ ]:

# --------------------------- IMPORTS ----------------------------------
import random, pathlib, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --------------------------- CONSTANTS --------------------------------
CSV_PATH   = "merged_8_july_d_lowmassremoved.csv"          # master dataset
BASE_OUT   = pathlib.Path("results_lam0p1")   # parent folder for all isotopes
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

# ---- fixed hyper-parameters (λ = 0 run) ----
LAM        = 0.1
N_HIDDEN   = 160
N_LAYERS   = 3
DROPOUT    = 0.05975904468734236
LR         = 0.0013783694458205406

MAX_EPOCHS = 200
PATIENCE   = 15
BATCH_SHUFFLE = True

N_RUNS     = 10
SEED_BASE  = 1234
EPS        = 1e-12

FEATURES = [
    'ERG','Radius','n_rms_radius','p_rms_radius','octopole_deformation',
    'n_chem_erg','Sn','Z','S2p_compound','gamma_deformation','ME',
    'BEA_A_daughter','A','Radius_daughter','BEA_A','AM','Radius_compound',
    'Pairing_daughter','BEA_compound','beta_deformation','Theory_thresh',
    'shell_P','pairing_delta','symmetry_S','excitation_energy',
    'Thresh_n3n','Excite_n3n','Thresh_n4n','Excite_n4n'
]
TARGET, SIGMA_EXP, FLUENCE, GAMMA, ISO = (
    "XS","sigma_exp","fluence","gamma","iso_id"
)

# --------------------------- UTILITIES --------------------------------
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ----------------------- DATASET & SAMPLER ----------------------------
class XSData(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler,
                 fit_scaler: bool = False):
        if fit_scaler:
            scaler.fit(df[FEATURES])
        self.X = torch.tensor(scaler.transform(df[FEATURES]).astype(np.float32))
        self.y = torch.tensor(df[TARGET].to_numpy(np.float32))
        self.fluence   = torch.tensor(df[FLUENCE].to_numpy(np.float32))
        self.sigma_exp = torch.tensor(df[SIGMA_EXP].fillna(0).to_numpy(np.float32))
        self.gamma     = torch.tensor(df[GAMMA].to_numpy(np.float32))
        self.iso_code  = df[ISO].astype("category").cat.codes.to_numpy()

    def __len__(self): return len(self.y)

    def __getitem__(self, idx: int):
        return {"x": self.X[idx], "y": self.y[idx],
                "fluence": self.fluence[idx],
                "sigma_exp": self.sigma_exp[idx],
                "gamma": self.gamma[idx],
                "iso_code": self.iso_code[idx]}

class IsoBatchSampler(Sampler):
    def __init__(self, iso_codes, shuffle=True):
        self.indices = {}
        for i, iso in enumerate(iso_codes):
            self.indices.setdefault(int(iso), []).append(i)
        self.keys = list(self.indices.keys())
        self.shuffle = shuffle

    def __iter__(self):
        keys = self.keys.copy()
        if self.shuffle: random.shuffle(keys)
        for k in keys:
            yield self.indices[k]

    def __len__(self): return len(self.keys)

# ------------------------------ MODEL ---------------------------------
class MLP(nn.Module):
    def __init__(self, n_in:int, n_hidden:int, n_layers:int, p_drop:float):
        super().__init__()
        layers = []
        for l in range(n_layers):
            layers += [nn.Linear(n_in if l==0 else n_hidden, n_hidden),
                       nn.ReLU(), nn.Dropout(p_drop)]
        layers.append(nn.Linear(n_hidden, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x).squeeze(1)

# -------------------------- CUSTOM LOSS -------------------------------
def iso_loss(batch, preds, lam:float):
    mse = nn.functional.mse_loss(preds, batch["y"])
    if batch["gamma"][0] == 0:     # skip σ₁g term if γ = 0
        return mse
    flu        = batch["fluence"]
    sigma_pred = (flu * preds).sum() / (flu.sum() + EPS)
    sigma_true = batch["sigma_exp"][0]
    rel_err_sq = ((sigma_pred - sigma_true) / (sigma_true + EPS))**2
    return mse + lam * rel_err_sq

# ------------------------ TRAIN / VALID LOOP --------------------------
def run_epoch(model, loader, opt, lam):
    is_train = opt is not None
    model.train(is_train)
    total, n = 0.0, 0
    for batch in loader:
        batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                 for k,v in batch.items()}
        preds = model(batch["x"])
        loss  = iso_loss(batch, preds, lam)
        if is_train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        total += loss.item() * len(preds)
        n     += len(preds)
    return total / n

# ----------------------- PER-ISOTOPE ROUTINE --------------------------
def process_isotope(df_all: pd.DataFrame, iso_hold: str) -> None:
    out_dir = BASE_OUT / iso_hold
    out_dir.mkdir(parents=True, exist_ok=True)

    df_test  = df_all[df_all[ISO] == iso_hold].reset_index(drop=True)
    df_train = df_all[df_all[ISO] != iso_hold].reset_index(drop=True)
    if df_test.empty:
        print(f"⚠  No rows for iso_id={iso_hold}; skipping.")
        return

    preds_runs = []
    for run in range(N_RUNS):
        set_seed(SEED_BASE + run)

        scaler = StandardScaler()
        ds_tr  = XSData(df_train, scaler, fit_scaler=True)
        ds_te  = XSData(df_test,  scaler, fit_scaler=False)

        dl_tr = DataLoader(ds_tr, batch_sampler=IsoBatchSampler(ds_tr.iso_code,
                                                                shuffle=BATCH_SHUFFLE))
        dl_te = DataLoader(ds_te, batch_sampler=IsoBatchSampler(ds_te.iso_code,
                                                                shuffle=False))

        model = MLP(len(FEATURES), N_HIDDEN, N_LAYERS, DROPOUT).to(DEVICE)
        opt   = torch.optim.Adam(model.parameters(), lr=LR)

        best, patience = float("inf"), 0
        for _ in range(MAX_EPOCHS):
            loss_tr = run_epoch(model, dl_tr, opt, LAM)
            if loss_tr < best - 1e-6:
                best, patience = loss_tr, 0
            else:
                patience += 1
                if patience >= PATIENCE:
                    break

        # predict
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in dl_te:
                batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                         for k,v in batch.items()}
                preds.append(model(batch["x"]).cpu())
        preds_runs.append(torch.cat(preds).numpy())

    preds_arr  = np.stack(preds_runs, axis=1)
    pred_mean  = preds_arr.mean(axis=1)

    # save CSV ----------------------------------------------------------
    df_out = pd.DataFrame({"ERG": df_test["ERG"], "XS_true": df_test["XS"]})
    for r in range(N_RUNS):
        df_out[f"pred_run{r+1}"] = preds_arr[:, r]
    df_out["pred_mean"] = pred_mean
    csv_path = out_dir / f"{iso_hold}_predictions.csv"
    df_out.to_csv(csv_path, index=False)

    # plot --------------------------------------------------------------
    plt.figure(figsize=(6,4))
    plt.plot(df_out["ERG"], df_out["XS_true"],  lw=2, label="True XS")
    plt.plot(df_out["ERG"], df_out["pred_mean"], lw=2, ls="--",
             label=f"Mean pred ({N_RUNS}×)")
    plt.xlabel("ERG"); plt.ylabel("XS")
    plt.title(f"{iso_hold} – true vs predicted XS")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / f"{iso_hold}_erg_vs_pred.png", dpi=300)
    plt.close()

    # meta --------------------------------------------------------------
    meta = dict(lam=LAM, n_hidden=N_HIDDEN, n_layers=N_LAYERS,
                dropout=DROPOUT, lr=LR, n_runs=N_RUNS,
                iso_held_out=iso_hold)
    with open(out_dir / "run_meta.json", "w") as fh:
        json.dump(meta, fh, indent=2)

    print(f"✓ finished {iso_hold}")

# ------------------------------- MAIN ---------------------------------
def main() -> None:
    BASE_OUT.mkdir(parents=True, exist_ok=True)
    df_all   = pd.read_csv(CSV_PATH)
    iso_list = sorted(df_all[ISO].unique())

    print(f"Found {len(iso_list)} unique isotopes.")
    for iso in iso_list:
        process_isotope(df_all, iso)

    print("\nAll isotopes done!")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()

In [ ]:

# --------------------------- IMPORTS ----------------------------------
import random, pathlib, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --------------------------- CONSTANTS --------------------------------
CSV_PATH   = "merged_8_july_d_lowmassremoved.csv"          # master dataset
BASE_OUT   = pathlib.Path("results_lam0p2")   # parent folder for all isotopes
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

# ---- fixed hyper-parameters (λ = 0 run) ----
LAM        = 0.2
N_HIDDEN   = 32
N_LAYERS   = 4
DROPOUT    = 0.00043729211213425906
LR         = 0.0006493008071290138

MAX_EPOCHS = 200
PATIENCE   = 15
BATCH_SHUFFLE = True

N_RUNS     = 10
SEED_BASE  = 1234
EPS        = 1e-12

FEATURES = [
    'ERG','Radius','n_rms_radius','p_rms_radius','octopole_deformation',
    'n_chem_erg','Sn','Z','S2p_compound','gamma_deformation','ME',
    'BEA_A_daughter','A','Radius_daughter','BEA_A','AM','Radius_compound',
    'Pairing_daughter','BEA_compound','beta_deformation','Theory_thresh',
    'shell_P','pairing_delta','symmetry_S','excitation_energy',
    'Thresh_n3n','Excite_n3n','Thresh_n4n','Excite_n4n'
]
TARGET, SIGMA_EXP, FLUENCE, GAMMA, ISO = (
    "XS","sigma_exp","fluence","gamma","iso_id"
)

# --------------------------- UTILITIES --------------------------------
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ----------------------- DATASET & SAMPLER ----------------------------
class XSData(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler,
                 fit_scaler: bool = False):
        if fit_scaler:
            scaler.fit(df[FEATURES])
        self.X = torch.tensor(scaler.transform(df[FEATURES]).astype(np.float32))
        self.y = torch.tensor(df[TARGET].to_numpy(np.float32))
        self.fluence   = torch.tensor(df[FLUENCE].to_numpy(np.float32))
        self.sigma_exp = torch.tensor(df[SIGMA_EXP].fillna(0).to_numpy(np.float32))
        self.gamma     = torch.tensor(df[GAMMA].to_numpy(np.float32))
        self.iso_code  = df[ISO].astype("category").cat.codes.to_numpy()

    def __len__(self): return len(self.y)

    def __getitem__(self, idx: int):
        return {"x": self.X[idx], "y": self.y[idx],
                "fluence": self.fluence[idx],
                "sigma_exp": self.sigma_exp[idx],
                "gamma": self.gamma[idx],
                "iso_code": self.iso_code[idx]}

class IsoBatchSampler(Sampler):
    def __init__(self, iso_codes, shuffle=True):
        self.indices = {}
        for i, iso in enumerate(iso_codes):
            self.indices.setdefault(int(iso), []).append(i)
        self.keys = list(self.indices.keys())
        self.shuffle = shuffle

    def __iter__(self):
        keys = self.keys.copy()
        if self.shuffle: random.shuffle(keys)
        for k in keys:
            yield self.indices[k]

    def __len__(self): return len(self.keys)

# ------------------------------ MODEL ---------------------------------
class MLP(nn.Module):
    def __init__(self, n_in:int, n_hidden:int, n_layers:int, p_drop:float):
        super().__init__()
        layers = []
        for l in range(n_layers):
            layers += [nn.Linear(n_in if l==0 else n_hidden, n_hidden),
                       nn.ReLU(), nn.Dropout(p_drop)]
        layers.append(nn.Linear(n_hidden, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x).squeeze(1)

# -------------------------- CUSTOM LOSS -------------------------------
def iso_loss(batch, preds, lam:float):
    mse = nn.functional.mse_loss(preds, batch["y"])
    if batch["gamma"][0] == 0:     # skip σ₁g term if γ = 0
        return mse
    flu        = batch["fluence"]
    sigma_pred = (flu * preds).sum() / (flu.sum() + EPS)
    sigma_true = batch["sigma_exp"][0]
    rel_err_sq = ((sigma_pred - sigma_true) / (sigma_true + EPS))**2
    return mse + lam * rel_err_sq

# ------------------------ TRAIN / VALID LOOP --------------------------
def run_epoch(model, loader, opt, lam):
    is_train = opt is not None
    model.train(is_train)
    total, n = 0.0, 0
    for batch in loader:
        batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                 for k,v in batch.items()}
        preds = model(batch["x"])
        loss  = iso_loss(batch, preds, lam)
        if is_train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        total += loss.item() * len(preds)
        n     += len(preds)
    return total / n

# ----------------------- PER-ISOTOPE ROUTINE --------------------------
def process_isotope(df_all: pd.DataFrame, iso_hold: str) -> None:
    out_dir = BASE_OUT / iso_hold
    out_dir.mkdir(parents=True, exist_ok=True)

    df_test  = df_all[df_all[ISO] == iso_hold].reset_index(drop=True)
    df_train = df_all[df_all[ISO] != iso_hold].reset_index(drop=True)
    if df_test.empty:
        print(f"⚠  No rows for iso_id={iso_hold}; skipping.")
        return

    preds_runs = []
    for run in range(N_RUNS):
        set_seed(SEED_BASE + run)

        scaler = StandardScaler()
        ds_tr  = XSData(df_train, scaler, fit_scaler=True)
        ds_te  = XSData(df_test,  scaler, fit_scaler=False)

        dl_tr = DataLoader(ds_tr, batch_sampler=IsoBatchSampler(ds_tr.iso_code,
                                                                shuffle=BATCH_SHUFFLE))
        dl_te = DataLoader(ds_te, batch_sampler=IsoBatchSampler(ds_te.iso_code,
                                                                shuffle=False))

        model = MLP(len(FEATURES), N_HIDDEN, N_LAYERS, DROPOUT).to(DEVICE)
        opt   = torch.optim.Adam(model.parameters(), lr=LR)

        best, patience = float("inf"), 0
        for _ in range(MAX_EPOCHS):
            loss_tr = run_epoch(model, dl_tr, opt, LAM)
            if loss_tr < best - 1e-6:
                best, patience = loss_tr, 0
            else:
                patience += 1
                if patience >= PATIENCE:
                    break

        # predict
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in dl_te:
                batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                         for k,v in batch.items()}
                preds.append(model(batch["x"]).cpu())
        preds_runs.append(torch.cat(preds).numpy())

    preds_arr  = np.stack(preds_runs, axis=1)
    pred_mean  = preds_arr.mean(axis=1)

    # save CSV ----------------------------------------------------------
    df_out = pd.DataFrame({"ERG": df_test["ERG"], "XS_true": df_test["XS"]})
    for r in range(N_RUNS):
        df_out[f"pred_run{r+1}"] = preds_arr[:, r]
    df_out["pred_mean"] = pred_mean
    csv_path = out_dir / f"{iso_hold}_predictions.csv"
    df_out.to_csv(csv_path, index=False)

    # plot --------------------------------------------------------------
    plt.figure(figsize=(6,4))
    plt.plot(df_out["ERG"], df_out["XS_true"],  lw=2, label="True XS")
    plt.plot(df_out["ERG"], df_out["pred_mean"], lw=2, ls="--",
             label=f"Mean pred ({N_RUNS}×)")
    plt.xlabel("ERG"); plt.ylabel("XS")
    plt.title(f"{iso_hold} – true vs predicted XS")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / f"{iso_hold}_erg_vs_pred.png", dpi=300)
    plt.close()

    # meta --------------------------------------------------------------
    meta = dict(lam=LAM, n_hidden=N_HIDDEN, n_layers=N_LAYERS,
                dropout=DROPOUT, lr=LR, n_runs=N_RUNS,
                iso_held_out=iso_hold)
    with open(out_dir / "run_meta.json", "w") as fh:
        json.dump(meta, fh, indent=2)

    print(f"✓ finished {iso_hold}")

# ------------------------------- MAIN ---------------------------------
def main() -> None:
    BASE_OUT.mkdir(parents=True, exist_ok=True)
    df_all   = pd.read_csv(CSV_PATH)
    iso_list = sorted(df_all[ISO].unique())

    print(f"Found {len(iso_list)} unique isotopes.")
    for iso in iso_list:
        process_isotope(df_all, iso)

    print("\nAll isotopes done!")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()

In [ ]:

# --------------------------- IMPORTS ----------------------------------
import random, pathlib, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --------------------------- CONSTANTS --------------------------------
CSV_PATH   = "merged_8_july_d_lowmassremoved.csv"          # master dataset
BASE_OUT   = pathlib.Path("results_lam0p3")   # parent folder for all isotopes
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"


LAM        = 0.3
N_HIDDEN   = 96
N_LAYERS   = 2
DROPOUT    = 0.08283102638417343
LR         = 0.0013388761576798537

MAX_EPOCHS = 200
PATIENCE   = 15
BATCH_SHUFFLE = True

N_RUNS     = 10
SEED_BASE  = 1234
EPS        = 1e-12

FEATURES = [
    'ERG','Radius','n_rms_radius','p_rms_radius','octopole_deformation',
    'n_chem_erg','Sn','Z','S2p_compound','gamma_deformation','ME',
    'BEA_A_daughter','A','Radius_daughter','BEA_A','AM','Radius_compound',
    'Pairing_daughter','BEA_compound','beta_deformation','Theory_thresh',
    'shell_P','pairing_delta','symmetry_S','excitation_energy',
    'Thresh_n3n','Excite_n3n','Thresh_n4n','Excite_n4n'
]
TARGET, SIGMA_EXP, FLUENCE, GAMMA, ISO = (
    "XS","sigma_exp","fluence","gamma","iso_id"
)

# --------------------------- UTILITIES --------------------------------
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ----------------------- DATASET & SAMPLER ----------------------------
class XSData(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler,
                 fit_scaler: bool = False):
        if fit_scaler:
            scaler.fit(df[FEATURES])
        self.X = torch.tensor(scaler.transform(df[FEATURES]).astype(np.float32))
        self.y = torch.tensor(df[TARGET].to_numpy(np.float32))
        self.fluence   = torch.tensor(df[FLUENCE].to_numpy(np.float32))
        self.sigma_exp = torch.tensor(df[SIGMA_EXP].fillna(0).to_numpy(np.float32))
        self.gamma     = torch.tensor(df[GAMMA].to_numpy(np.float32))
        self.iso_code  = df[ISO].astype("category").cat.codes.to_numpy()

    def __len__(self): return len(self.y)

    def __getitem__(self, idx: int):
        return {"x": self.X[idx], "y": self.y[idx],
                "fluence": self.fluence[idx],
                "sigma_exp": self.sigma_exp[idx],
                "gamma": self.gamma[idx],
                "iso_code": self.iso_code[idx]}

class IsoBatchSampler(Sampler):
    def __init__(self, iso_codes, shuffle=True):
        self.indices = {}
        for i, iso in enumerate(iso_codes):
            self.indices.setdefault(int(iso), []).append(i)
        self.keys = list(self.indices.keys())
        self.shuffle = shuffle

    def __iter__(self):
        keys = self.keys.copy()
        if self.shuffle: random.shuffle(keys)
        for k in keys:
            yield self.indices[k]

    def __len__(self): return len(self.keys)

# ------------------------------ MODEL ---------------------------------
class MLP(nn.Module):
    def __init__(self, n_in:int, n_hidden:int, n_layers:int, p_drop:float):
        super().__init__()
        layers = []
        for l in range(n_layers):
            layers += [nn.Linear(n_in if l==0 else n_hidden, n_hidden),
                       nn.ReLU(), nn.Dropout(p_drop)]
        layers.append(nn.Linear(n_hidden, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x).squeeze(1)

# -------------------------- CUSTOM LOSS -------------------------------
def iso_loss(batch, preds, lam:float):
    mse = nn.functional.mse_loss(preds, batch["y"])
    if batch["gamma"][0] == 0:     # skip σ₁g term if γ = 0
        return mse
    flu        = batch["fluence"]
    sigma_pred = (flu * preds).sum() / (flu.sum() + EPS)
    sigma_true = batch["sigma_exp"][0]
    rel_err_sq = ((sigma_pred - sigma_true) / (sigma_true + EPS))**2
    return mse + lam * rel_err_sq

# ------------------------ TRAIN / VALID LOOP --------------------------
def run_epoch(model, loader, opt, lam):
    is_train = opt is not None
    model.train(is_train)
    total, n = 0.0, 0
    for batch in loader:
        batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                 for k,v in batch.items()}
        preds = model(batch["x"])
        loss  = iso_loss(batch, preds, lam)
        if is_train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        total += loss.item() * len(preds)
        n     += len(preds)
    return total / n

# ----------------------- PER-ISOTOPE ROUTINE --------------------------
def process_isotope(df_all: pd.DataFrame, iso_hold: str) -> None:
    out_dir = BASE_OUT / iso_hold
    out_dir.mkdir(parents=True, exist_ok=True)

    df_test  = df_all[df_all[ISO] == iso_hold].reset_index(drop=True)
    df_train = df_all[df_all[ISO] != iso_hold].reset_index(drop=True)
    if df_test.empty:
        print(f"⚠  No rows for iso_id={iso_hold}; skipping.")
        return

    preds_runs = []
    for run in range(N_RUNS):
        set_seed(SEED_BASE + run)

        scaler = StandardScaler()
        ds_tr  = XSData(df_train, scaler, fit_scaler=True)
        ds_te  = XSData(df_test,  scaler, fit_scaler=False)

        dl_tr = DataLoader(ds_tr, batch_sampler=IsoBatchSampler(ds_tr.iso_code,
                                                                shuffle=BATCH_SHUFFLE))
        dl_te = DataLoader(ds_te, batch_sampler=IsoBatchSampler(ds_te.iso_code,
                                                                shuffle=False))

        model = MLP(len(FEATURES), N_HIDDEN, N_LAYERS, DROPOUT).to(DEVICE)
        opt   = torch.optim.Adam(model.parameters(), lr=LR)

        best, patience = float("inf"), 0
        for _ in range(MAX_EPOCHS):
            loss_tr = run_epoch(model, dl_tr, opt, LAM)
            if loss_tr < best - 1e-6:
                best, patience = loss_tr, 0
            else:
                patience += 1
                if patience >= PATIENCE:
                    break

        # predict
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in dl_te:
                batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                         for k,v in batch.items()}
                preds.append(model(batch["x"]).cpu())
        preds_runs.append(torch.cat(preds).numpy())

    preds_arr  = np.stack(preds_runs, axis=1)
    pred_mean  = preds_arr.mean(axis=1)

    # save CSV ----------------------------------------------------------
    df_out = pd.DataFrame({"ERG": df_test["ERG"], "XS_true": df_test["XS"]})
    for r in range(N_RUNS):
        df_out[f"pred_run{r+1}"] = preds_arr[:, r]
    df_out["pred_mean"] = pred_mean
    csv_path = out_dir / f"{iso_hold}_predictions.csv"
    df_out.to_csv(csv_path, index=False)

    # plot --------------------------------------------------------------
    plt.figure(figsize=(6,4))
    plt.plot(df_out["ERG"], df_out["XS_true"],  lw=2, label="True XS")
    plt.plot(df_out["ERG"], df_out["pred_mean"], lw=2, ls="--",
             label=f"Mean pred ({N_RUNS}×)")
    plt.xlabel("ERG"); plt.ylabel("XS")
    plt.title(f"{iso_hold} – true vs predicted XS")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / f"{iso_hold}_erg_vs_pred.png", dpi=300)
    plt.close()

    # meta --------------------------------------------------------------
    meta = dict(lam=LAM, n_hidden=N_HIDDEN, n_layers=N_LAYERS,
                dropout=DROPOUT, lr=LR, n_runs=N_RUNS,
                iso_held_out=iso_hold)
    with open(out_dir / "run_meta.json", "w") as fh:
        json.dump(meta, fh, indent=2)

    print(f"✓ finished {iso_hold}")

# ------------------------------- MAIN ---------------------------------
def main() -> None:
    BASE_OUT.mkdir(parents=True, exist_ok=True)
    df_all   = pd.read_csv(CSV_PATH)
    iso_list = sorted(df_all[ISO].unique())

    print(f"Found {len(iso_list)} unique isotopes.")
    for iso in iso_list:
        process_isotope(df_all, iso)

    print("\nAll isotopes done!")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()

In [ ]:
# --------------------------- IMPORTS ----------------------------------
import random, pathlib, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --------------------------- CONSTANTS --------------------------------
CSV_PATH   = "merged_8_july_d_lowmassremoved.csv"          # master dataset
BASE_OUT   = pathlib.Path("results_lam0p4")   # parent folder for all isotopes
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"


LAM        = 0.4
N_HIDDEN   = 96
N_LAYERS   = 2
DROPOUT    = 0.032508366621761715
LR         = 0.0006684238765748816

MAX_EPOCHS = 200
PATIENCE   = 15
BATCH_SHUFFLE = True

N_RUNS     = 10
SEED_BASE  = 1234
EPS        = 1e-12

FEATURES = [
    'ERG','Radius','n_rms_radius','p_rms_radius','octopole_deformation',
    'n_chem_erg','Sn','Z','S2p_compound','gamma_deformation','ME',
    'BEA_A_daughter','A','Radius_daughter','BEA_A','AM','Radius_compound',
    'Pairing_daughter','BEA_compound','beta_deformation','Theory_thresh',
    'shell_P','pairing_delta','symmetry_S','excitation_energy',
    'Thresh_n3n','Excite_n3n','Thresh_n4n','Excite_n4n'
]
TARGET, SIGMA_EXP, FLUENCE, GAMMA, ISO = (
    "XS","sigma_exp","fluence","gamma","iso_id"
)

# --------------------------- UTILITIES --------------------------------
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ----------------------- DATASET & SAMPLER ----------------------------
class XSData(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler,
                 fit_scaler: bool = False):
        if fit_scaler:
            scaler.fit(df[FEATURES])
        self.X = torch.tensor(scaler.transform(df[FEATURES]).astype(np.float32))
        self.y = torch.tensor(df[TARGET].to_numpy(np.float32))
        self.fluence   = torch.tensor(df[FLUENCE].to_numpy(np.float32))
        self.sigma_exp = torch.tensor(df[SIGMA_EXP].fillna(0).to_numpy(np.float32))
        self.gamma     = torch.tensor(df[GAMMA].to_numpy(np.float32))
        self.iso_code  = df[ISO].astype("category").cat.codes.to_numpy()

    def __len__(self): return len(self.y)

    def __getitem__(self, idx: int):
        return {"x": self.X[idx], "y": self.y[idx],
                "fluence": self.fluence[idx],
                "sigma_exp": self.sigma_exp[idx],
                "gamma": self.gamma[idx],
                "iso_code": self.iso_code[idx]}

class IsoBatchSampler(Sampler):
    def __init__(self, iso_codes, shuffle=True):
        self.indices = {}
        for i, iso in enumerate(iso_codes):
            self.indices.setdefault(int(iso), []).append(i)
        self.keys = list(self.indices.keys())
        self.shuffle = shuffle

    def __iter__(self):
        keys = self.keys.copy()
        if self.shuffle: random.shuffle(keys)
        for k in keys:
            yield self.indices[k]

    def __len__(self): return len(self.keys)

# ------------------------------ MODEL ---------------------------------
class MLP(nn.Module):
    def __init__(self, n_in:int, n_hidden:int, n_layers:int, p_drop:float):
        super().__init__()
        layers = []
        for l in range(n_layers):
            layers += [nn.Linear(n_in if l==0 else n_hidden, n_hidden),
                       nn.ReLU(), nn.Dropout(p_drop)]
        layers.append(nn.Linear(n_hidden, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x).squeeze(1)

# -------------------------- CUSTOM LOSS -------------------------------
def iso_loss(batch, preds, lam:float):
    mse = nn.functional.mse_loss(preds, batch["y"])
    if batch["gamma"][0] == 0:     # skip σ₁g term if γ = 0
        return mse
    flu        = batch["fluence"]
    sigma_pred = (flu * preds).sum() / (flu.sum() + EPS)
    sigma_true = batch["sigma_exp"][0]
    rel_err_sq = ((sigma_pred - sigma_true) / (sigma_true + EPS))**2
    return mse + lam * rel_err_sq

# ------------------------ TRAIN / VALID LOOP --------------------------
def run_epoch(model, loader, opt, lam):
    is_train = opt is not None
    model.train(is_train)
    total, n = 0.0, 0
    for batch in loader:
        batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                 for k,v in batch.items()}
        preds = model(batch["x"])
        loss  = iso_loss(batch, preds, lam)
        if is_train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        total += loss.item() * len(preds)
        n     += len(preds)
    return total / n

# ----------------------- PER-ISOTOPE ROUTINE --------------------------
def process_isotope(df_all: pd.DataFrame, iso_hold: str) -> None:
    out_dir = BASE_OUT / iso_hold
    out_dir.mkdir(parents=True, exist_ok=True)

    df_test  = df_all[df_all[ISO] == iso_hold].reset_index(drop=True)
    df_train = df_all[df_all[ISO] != iso_hold].reset_index(drop=True)
    if df_test.empty:
        print(f"⚠  No rows for iso_id={iso_hold}; skipping.")
        return

    preds_runs = []
    for run in range(N_RUNS):
        set_seed(SEED_BASE + run)

        scaler = StandardScaler()
        ds_tr  = XSData(df_train, scaler, fit_scaler=True)
        ds_te  = XSData(df_test,  scaler, fit_scaler=False)

        dl_tr = DataLoader(ds_tr, batch_sampler=IsoBatchSampler(ds_tr.iso_code,
                                                                shuffle=BATCH_SHUFFLE))
        dl_te = DataLoader(ds_te, batch_sampler=IsoBatchSampler(ds_te.iso_code,
                                                                shuffle=False))

        model = MLP(len(FEATURES), N_HIDDEN, N_LAYERS, DROPOUT).to(DEVICE)
        opt   = torch.optim.Adam(model.parameters(), lr=LR)

        best, patience = float("inf"), 0
        for _ in range(MAX_EPOCHS):
            loss_tr = run_epoch(model, dl_tr, opt, LAM)
            if loss_tr < best - 1e-6:
                best, patience = loss_tr, 0
            else:
                patience += 1
                if patience >= PATIENCE:
                    break

        # predict
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in dl_te:
                batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                         for k,v in batch.items()}
                preds.append(model(batch["x"]).cpu())
        preds_runs.append(torch.cat(preds).numpy())

    preds_arr  = np.stack(preds_runs, axis=1)
    pred_mean  = preds_arr.mean(axis=1)

    # save CSV ----------------------------------------------------------
    df_out = pd.DataFrame({"ERG": df_test["ERG"], "XS_true": df_test["XS"]})
    for r in range(N_RUNS):
        df_out[f"pred_run{r+1}"] = preds_arr[:, r]
    df_out["pred_mean"] = pred_mean
    csv_path = out_dir / f"{iso_hold}_predictions.csv"
    df_out.to_csv(csv_path, index=False)

    # plot --------------------------------------------------------------
    plt.figure(figsize=(6,4))
    plt.plot(df_out["ERG"], df_out["XS_true"],  lw=2, label="True XS")
    plt.plot(df_out["ERG"], df_out["pred_mean"], lw=2, ls="--",
             label=f"Mean pred ({N_RUNS}×)")
    plt.xlabel("ERG"); plt.ylabel("XS")
    plt.title(f"{iso_hold} – true vs predicted XS")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / f"{iso_hold}_erg_vs_pred.png", dpi=300)
    plt.close()

    # meta --------------------------------------------------------------
    meta = dict(lam=LAM, n_hidden=N_HIDDEN, n_layers=N_LAYERS,
                dropout=DROPOUT, lr=LR, n_runs=N_RUNS,
                iso_held_out=iso_hold)
    with open(out_dir / "run_meta.json", "w") as fh:
        json.dump(meta, fh, indent=2)

    print(f"✓ finished {iso_hold}")

# ------------------------------- MAIN ---------------------------------
def main() -> None:
    BASE_OUT.mkdir(parents=True, exist_ok=True)
    df_all   = pd.read_csv(CSV_PATH)
    iso_list = sorted(df_all[ISO].unique())

    print(f"Found {len(iso_list)} unique isotopes.")
    for iso in iso_list:
        process_isotope(df_all, iso)

    print("\nAll isotopes done!")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()

In [ ]:


# --------------------------- IMPORTS ----------------------------------
import os, random, pathlib, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --------------------------- CONSTANTS --------------------------------
CSV_PATH   = "merged_8_july_d_lowmassremoved.csv"       # master data file
BASE_OUT   = pathlib.Path("results_lam0p5")  # parent folder for all isotopes
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

LAM        = 0.5
N_HIDDEN   = 160
N_LAYERS   = 4
DROPOUT    = 0.00811351363377738
LR         = 1.6705487638549014e-4

MAX_EPOCHS = 200
PATIENCE   = 15
BATCH_SHUFFLE = True

N_RUNS     = 10
SEED_BASE  = 1234
EPS        = 1e-12

FEATURES = [
    'ERG','Radius','n_rms_radius','p_rms_radius','octopole_deformation',
    'n_chem_erg','Sn','Z','S2p_compound','gamma_deformation','ME',
    'BEA_A_daughter','A','Radius_daughter','BEA_A','AM','Radius_compound',
    'Pairing_daughter','BEA_compound','beta_deformation','Theory_thresh',
    'shell_P','pairing_delta','symmetry_S','excitation_energy',
    'Thresh_n3n','Excite_n3n','Thresh_n4n','Excite_n4n'
]
TARGET, SIGMA_EXP, FLUENCE, GAMMA, ISO = (
    "XS","sigma_exp","fluence","gamma","iso_id"
)

# --------------------------- UTILITIES --------------------------------
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ----------------------- DATASET & SAMPLER ----------------------------
class XSData(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler,
                 fit_scaler: bool = False):
        if fit_scaler:
            scaler.fit(df[FEATURES])
        self.X = torch.tensor(scaler.transform(df[FEATURES]).astype(np.float32))
        self.y = torch.tensor(df[TARGET].to_numpy(np.float32))
        self.fluence    = torch.tensor(df[FLUENCE].to_numpy(np.float32))
        self.sigma_exp  = torch.tensor(df[SIGMA_EXP].fillna(0).to_numpy(np.float32))
        self.gamma      = torch.tensor(df[GAMMA].to_numpy(np.float32))
        self.iso_code   = df[ISO].astype("category").cat.codes.to_numpy()

    def __len__(self): return len(self.y)

    def __getitem__(self, idx):
        return {
            "x":          self.X[idx],
            "y":          self.y[idx],
            "fluence":    self.fluence[idx],
            "sigma_exp":  self.sigma_exp[idx],
            "gamma":      self.gamma[idx],
            "iso_code":   self.iso_code[idx]
        }

class IsoBatchSampler(Sampler):
    def __init__(self, iso_codes, shuffle=True):
        self.indices = {}
        for i, iso in enumerate(iso_codes):
            self.indices.setdefault(int(iso), []).append(i)
        self.keys = list(self.indices.keys())
        self.shuffle = shuffle

    def __iter__(self):
        keys = self.keys.copy()
        if self.shuffle: random.shuffle(keys)
        for k in keys:
            yield self.indices[k]

    def __len__(self): return len(self.keys)

# ------------------------------ MODEL ---------------------------------
class MLP(nn.Module):
    def __init__(self, n_in, n_hidden, n_layers, p_drop):
        super().__init__()
        layers = []
        for l in range(n_layers):
            layers += [
                nn.Linear(n_in if l==0 else n_hidden, n_hidden),
                nn.ReLU(), nn.Dropout(p_drop)
            ]
        layers.append(nn.Linear(n_hidden, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x).squeeze(1)

# -------------------------- CUSTOM LOSS -------------------------------
def iso_loss(batch, preds, lam):
    mse = nn.functional.mse_loss(preds, batch["y"])
    if batch["gamma"][0] == 0:
        return mse
    flu         = batch["fluence"]
    sigma_pred  = (flu * preds).sum() / (flu.sum() + EPS)
    sigma_true  = batch["sigma_exp"][0]
    rel_err_sq  = ((sigma_pred - sigma_true) / (sigma_true + EPS)) ** 2
    return mse + lam * rel_err_sq

# ------------------------ TRAIN / VALID LOOP --------------------------
def run_epoch(model, loader, opt, lam):
    is_train = opt is not None
    model.train(is_train)
    total, n = 0.0, 0
    for batch in loader:
        batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                 for k,v in batch.items()}
        preds = model(batch["x"])
        loss  = iso_loss(batch, preds, lam)
        if is_train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        total += loss.item() * len(preds)
        n     += len(preds)
    return total / n

# ----------------------- PER-ISOTOPE PIPELINE -------------------------
def process_isotope(df_all: pd.DataFrame, iso_hold: str) -> None:
    out_dir = BASE_OUT / iso_hold
    out_dir.mkdir(parents=True, exist_ok=True)

    df_test  = df_all[df_all[ISO] == iso_hold].reset_index(drop=True)
    df_train = df_all[df_all[ISO] != iso_hold].reset_index(drop=True)
    if df_test.empty:
        print(f"⚠  No rows for iso_id={iso_hold}; skipping.")
        return

    all_preds = []
    for run in range(N_RUNS):
        set_seed(SEED_BASE + run)

        scaler = StandardScaler()
        ds_tr  = XSData(df_train, scaler, fit_scaler=True)
        ds_te  = XSData(df_test,  scaler, fit_scaler=False)

        dl_tr = DataLoader(ds_tr, batch_sampler=IsoBatchSampler(ds_tr.iso_code,
                                                                shuffle=BATCH_SHUFFLE))
        dl_te = DataLoader(ds_te, batch_sampler=IsoBatchSampler(ds_te.iso_code,
                                                                shuffle=False))

        model = MLP(len(FEATURES), N_HIDDEN, N_LAYERS, DROPOUT).to(DEVICE)
        opt   = torch.optim.Adam(model.parameters(), lr=LR)

        best, patience = float("inf"), 0
        for ep in range(MAX_EPOCHS):
            loss_tr = run_epoch(model, dl_tr, opt, LAM)
            if loss_tr < best - 1e-6:
                best, patience = loss_tr, 0
            else:
                patience += 1
                if patience >= PATIENCE: break

        # predict
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in dl_te:
                batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                         for k,v in batch.items()}
                preds.append(model(batch["x"]).cpu())
        all_preds.append(torch.cat(preds).numpy())

    preds_arr = np.stack(all_preds, axis=1)
    pred_mean = preds_arr.mean(axis=1)

    out_df = pd.DataFrame({"ERG": df_test["ERG"], "XS_true": df_test["XS"]})
    for r in range(N_RUNS):
        out_df[f"pred_run{r+1}"] = preds_arr[:, r]
    out_df["pred_mean"] = pred_mean

    csv_path = out_dir / f"{iso_hold}_predictions.csv"
    out_df.to_csv(csv_path, index=False)

    # plot
    plt.figure(figsize=(6,4))
    plt.plot(out_df["ERG"], out_df["XS_true"], lw=2, label="True XS")
    plt.plot(out_df["ERG"], out_df["pred_mean"], lw=2, ls="--",
             label=f"Mean pred ({N_RUNS}×)")
    plt.xlabel("ERG"); plt.ylabel("XS")
    plt.title(f"{iso_hold} – true vs predicted XS")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / f"{iso_hold}_erg_vs_pred.png", dpi=300)
    plt.close()

    # meta
    meta = dict(lam=LAM, n_hidden=N_HIDDEN, n_layers=N_LAYERS,
                dropout=DROPOUT, lr=LR, n_runs=N_RUNS,
                iso_held_out=iso_hold)
    with open(out_dir / "run_meta.json", "w") as fh:
        json.dump(meta, fh, indent=2)

    print(f"✓ finished {iso_hold}")

# ------------------------------- MAIN ---------------------------------
def main() -> None:
    BASE_OUT.mkdir(parents=True, exist_ok=True)
    df_all = pd.read_csv(CSV_PATH)
    iso_list = sorted(df_all[ISO].unique())

    print(f"Found {len(iso_list)} unique isotopes.")
    for iso in iso_list:
        process_isotope(df_all, iso)

    print("\nAll isotopes done!")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()


In [ ]:
# --------------------------- IMPORTS ----------------------------------
import random, pathlib, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --------------------------- CONSTANTS --------------------------------
CSV_PATH   = "merged_8_july_d_lowmassremoved.csv"          # master dataset
BASE_OUT   = pathlib.Path("results_lam0p6")   # parent folder for all isotopes
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"


LAM        = 0.6
N_HIDDEN   = 192
N_LAYERS   = 2
DROPOUT    = 0.0014696708413958846
LR         = 0.00046358470164797964

MAX_EPOCHS = 200
PATIENCE   = 15
BATCH_SHUFFLE = True

N_RUNS     = 10
SEED_BASE  = 1234
EPS        = 1e-12

FEATURES = [
    'ERG','Radius','n_rms_radius','p_rms_radius','octopole_deformation',
    'n_chem_erg','Sn','Z','S2p_compound','gamma_deformation','ME',
    'BEA_A_daughter','A','Radius_daughter','BEA_A','AM','Radius_compound',
    'Pairing_daughter','BEA_compound','beta_deformation','Theory_thresh',
    'shell_P','pairing_delta','symmetry_S','excitation_energy',
    'Thresh_n3n','Excite_n3n','Thresh_n4n','Excite_n4n'
]
TARGET, SIGMA_EXP, FLUENCE, GAMMA, ISO = (
    "XS","sigma_exp","fluence","gamma","iso_id"
)

# --------------------------- UTILITIES --------------------------------
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ----------------------- DATASET & SAMPLER ----------------------------
class XSData(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler,
                 fit_scaler: bool = False):
        if fit_scaler:
            scaler.fit(df[FEATURES])
        self.X = torch.tensor(scaler.transform(df[FEATURES]).astype(np.float32))
        self.y = torch.tensor(df[TARGET].to_numpy(np.float32))
        self.fluence   = torch.tensor(df[FLUENCE].to_numpy(np.float32))
        self.sigma_exp = torch.tensor(df[SIGMA_EXP].fillna(0).to_numpy(np.float32))
        self.gamma     = torch.tensor(df[GAMMA].to_numpy(np.float32))
        self.iso_code  = df[ISO].astype("category").cat.codes.to_numpy()

    def __len__(self): return len(self.y)

    def __getitem__(self, idx: int):
        return {"x": self.X[idx], "y": self.y[idx],
                "fluence": self.fluence[idx],
                "sigma_exp": self.sigma_exp[idx],
                "gamma": self.gamma[idx],
                "iso_code": self.iso_code[idx]}

class IsoBatchSampler(Sampler):
    def __init__(self, iso_codes, shuffle=True):
        self.indices = {}
        for i, iso in enumerate(iso_codes):
            self.indices.setdefault(int(iso), []).append(i)
        self.keys = list(self.indices.keys())
        self.shuffle = shuffle

    def __iter__(self):
        keys = self.keys.copy()
        if self.shuffle: random.shuffle(keys)
        for k in keys:
            yield self.indices[k]

    def __len__(self): return len(self.keys)

# ------------------------------ MODEL ---------------------------------
class MLP(nn.Module):
    def __init__(self, n_in:int, n_hidden:int, n_layers:int, p_drop:float):
        super().__init__()
        layers = []
        for l in range(n_layers):
            layers += [nn.Linear(n_in if l==0 else n_hidden, n_hidden),
                       nn.ReLU(), nn.Dropout(p_drop)]
        layers.append(nn.Linear(n_hidden, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x).squeeze(1)

# -------------------------- CUSTOM LOSS -------------------------------
def iso_loss(batch, preds, lam:float):
    mse = nn.functional.mse_loss(preds, batch["y"])
    if batch["gamma"][0] == 0:     # skip σ₁g term if γ = 0
        return mse
    flu        = batch["fluence"]
    sigma_pred = (flu * preds).sum() / (flu.sum() + EPS)
    sigma_true = batch["sigma_exp"][0]
    rel_err_sq = ((sigma_pred - sigma_true) / (sigma_true + EPS))**2
    return mse + lam * rel_err_sq

# ------------------------ TRAIN / VALID LOOP --------------------------
def run_epoch(model, loader, opt, lam):
    is_train = opt is not None
    model.train(is_train)
    total, n = 0.0, 0
    for batch in loader:
        batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                 for k,v in batch.items()}
        preds = model(batch["x"])
        loss  = iso_loss(batch, preds, lam)
        if is_train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        total += loss.item() * len(preds)
        n     += len(preds)
    return total / n

# ----------------------- PER-ISOTOPE ROUTINE --------------------------
def process_isotope(df_all: pd.DataFrame, iso_hold: str) -> None:
    out_dir = BASE_OUT / iso_hold
    out_dir.mkdir(parents=True, exist_ok=True)

    df_test  = df_all[df_all[ISO] == iso_hold].reset_index(drop=True)
    df_train = df_all[df_all[ISO] != iso_hold].reset_index(drop=True)
    if df_test.empty:
        print(f"⚠  No rows for iso_id={iso_hold}; skipping.")
        return

    preds_runs = []
    for run in range(N_RUNS):
        set_seed(SEED_BASE + run)

        scaler = StandardScaler()
        ds_tr  = XSData(df_train, scaler, fit_scaler=True)
        ds_te  = XSData(df_test,  scaler, fit_scaler=False)

        dl_tr = DataLoader(ds_tr, batch_sampler=IsoBatchSampler(ds_tr.iso_code,
                                                                shuffle=BATCH_SHUFFLE))
        dl_te = DataLoader(ds_te, batch_sampler=IsoBatchSampler(ds_te.iso_code,
                                                                shuffle=False))

        model = MLP(len(FEATURES), N_HIDDEN, N_LAYERS, DROPOUT).to(DEVICE)
        opt   = torch.optim.Adam(model.parameters(), lr=LR)

        best, patience = float("inf"), 0
        for _ in range(MAX_EPOCHS):
            loss_tr = run_epoch(model, dl_tr, opt, LAM)
            if loss_tr < best - 1e-6:
                best, patience = loss_tr, 0
            else:
                patience += 1
                if patience >= PATIENCE:
                    break

        # predict
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in dl_te:
                batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                         for k,v in batch.items()}
                preds.append(model(batch["x"]).cpu())
        preds_runs.append(torch.cat(preds).numpy())

    preds_arr  = np.stack(preds_runs, axis=1)
    pred_mean  = preds_arr.mean(axis=1)

    # save CSV ----------------------------------------------------------
    df_out = pd.DataFrame({"ERG": df_test["ERG"], "XS_true": df_test["XS"]})
    for r in range(N_RUNS):
        df_out[f"pred_run{r+1}"] = preds_arr[:, r]
    df_out["pred_mean"] = pred_mean
    csv_path = out_dir / f"{iso_hold}_predictions.csv"
    df_out.to_csv(csv_path, index=False)

    # plot --------------------------------------------------------------
    plt.figure(figsize=(6,4))
    plt.plot(df_out["ERG"], df_out["XS_true"],  lw=2, label="True XS")
    plt.plot(df_out["ERG"], df_out["pred_mean"], lw=2, ls="--",
             label=f"Mean pred ({N_RUNS}×)")
    plt.xlabel("ERG"); plt.ylabel("XS")
    plt.title(f"{iso_hold} – true vs predicted XS")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / f"{iso_hold}_erg_vs_pred.png", dpi=300)
    plt.close()

    # meta --------------------------------------------------------------
    meta = dict(lam=LAM, n_hidden=N_HIDDEN, n_layers=N_LAYERS,
                dropout=DROPOUT, lr=LR, n_runs=N_RUNS,
                iso_held_out=iso_hold)
    with open(out_dir / "run_meta.json", "w") as fh:
        json.dump(meta, fh, indent=2)

    print(f"✓ finished {iso_hold}")

# ------------------------------- MAIN ---------------------------------
def main() -> None:
    BASE_OUT.mkdir(parents=True, exist_ok=True)
    df_all   = pd.read_csv(CSV_PATH)
    iso_list = sorted(df_all[ISO].unique())

    print(f"Found {len(iso_list)} unique isotopes.")
    for iso in iso_list:
        process_isotope(df_all, iso)

    print("\nAll isotopes done!")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()

In [ ]:
# --------------------------- IMPORTS ----------------------------------
import random, pathlib, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --------------------------- CONSTANTS --------------------------------
CSV_PATH   = "merged_8_july_d_lowmassremoved.csv"          # master dataset
BASE_OUT   = pathlib.Path("results_lam0p7")   # parent folder for all isotopes
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"


LAM        = 0.7
N_HIDDEN   = 160
N_LAYERS   = 2
DROPOUT    = 0.011148537065922611
LR         = 0.0003653645325246077

MAX_EPOCHS = 200
PATIENCE   = 15
BATCH_SHUFFLE = True

N_RUNS     = 10
SEED_BASE  = 1234
EPS        = 1e-12

FEATURES = [
    'ERG','Radius','n_rms_radius','p_rms_radius','octopole_deformation',
    'n_chem_erg','Sn','Z','S2p_compound','gamma_deformation','ME',
    'BEA_A_daughter','A','Radius_daughter','BEA_A','AM','Radius_compound',
    'Pairing_daughter','BEA_compound','beta_deformation','Theory_thresh',
    'shell_P','pairing_delta','symmetry_S','excitation_energy',
    'Thresh_n3n','Excite_n3n','Thresh_n4n','Excite_n4n'
]
TARGET, SIGMA_EXP, FLUENCE, GAMMA, ISO = (
    "XS","sigma_exp","fluence","gamma","iso_id"
)

# --------------------------- UTILITIES --------------------------------
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ----------------------- DATASET & SAMPLER ----------------------------
class XSData(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler,
                 fit_scaler: bool = False):
        if fit_scaler:
            scaler.fit(df[FEATURES])
        self.X = torch.tensor(scaler.transform(df[FEATURES]).astype(np.float32))
        self.y = torch.tensor(df[TARGET].to_numpy(np.float32))
        self.fluence   = torch.tensor(df[FLUENCE].to_numpy(np.float32))
        self.sigma_exp = torch.tensor(df[SIGMA_EXP].fillna(0).to_numpy(np.float32))
        self.gamma     = torch.tensor(df[GAMMA].to_numpy(np.float32))
        self.iso_code  = df[ISO].astype("category").cat.codes.to_numpy()

    def __len__(self): return len(self.y)

    def __getitem__(self, idx: int):
        return {"x": self.X[idx], "y": self.y[idx],
                "fluence": self.fluence[idx],
                "sigma_exp": self.sigma_exp[idx],
                "gamma": self.gamma[idx],
                "iso_code": self.iso_code[idx]}

class IsoBatchSampler(Sampler):
    def __init__(self, iso_codes, shuffle=True):
        self.indices = {}
        for i, iso in enumerate(iso_codes):
            self.indices.setdefault(int(iso), []).append(i)
        self.keys = list(self.indices.keys())
        self.shuffle = shuffle

    def __iter__(self):
        keys = self.keys.copy()
        if self.shuffle: random.shuffle(keys)
        for k in keys:
            yield self.indices[k]

    def __len__(self): return len(self.keys)

# ------------------------------ MODEL ---------------------------------
class MLP(nn.Module):
    def __init__(self, n_in:int, n_hidden:int, n_layers:int, p_drop:float):
        super().__init__()
        layers = []
        for l in range(n_layers):
            layers += [nn.Linear(n_in if l==0 else n_hidden, n_hidden),
                       nn.ReLU(), nn.Dropout(p_drop)]
        layers.append(nn.Linear(n_hidden, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x).squeeze(1)

# -------------------------- CUSTOM LOSS -------------------------------
def iso_loss(batch, preds, lam:float):
    mse = nn.functional.mse_loss(preds, batch["y"])
    if batch["gamma"][0] == 0:     # skip σ₁g term if γ = 0
        return mse
    flu        = batch["fluence"]
    sigma_pred = (flu * preds).sum() / (flu.sum() + EPS)
    sigma_true = batch["sigma_exp"][0]
    rel_err_sq = ((sigma_pred - sigma_true) / (sigma_true + EPS))**2
    return mse + lam * rel_err_sq

# ------------------------ TRAIN / VALID LOOP --------------------------
def run_epoch(model, loader, opt, lam):
    is_train = opt is not None
    model.train(is_train)
    total, n = 0.0, 0
    for batch in loader:
        batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                 for k,v in batch.items()}
        preds = model(batch["x"])
        loss  = iso_loss(batch, preds, lam)
        if is_train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        total += loss.item() * len(preds)
        n     += len(preds)
    return total / n

# ----------------------- PER-ISOTOPE ROUTINE --------------------------
def process_isotope(df_all: pd.DataFrame, iso_hold: str) -> None:
    out_dir = BASE_OUT / iso_hold
    out_dir.mkdir(parents=True, exist_ok=True)

    df_test  = df_all[df_all[ISO] == iso_hold].reset_index(drop=True)
    df_train = df_all[df_all[ISO] != iso_hold].reset_index(drop=True)
    if df_test.empty:
        print(f"⚠  No rows for iso_id={iso_hold}; skipping.")
        return

    preds_runs = []
    for run in range(N_RUNS):
        set_seed(SEED_BASE + run)

        scaler = StandardScaler()
        ds_tr  = XSData(df_train, scaler, fit_scaler=True)
        ds_te  = XSData(df_test,  scaler, fit_scaler=False)

        dl_tr = DataLoader(ds_tr, batch_sampler=IsoBatchSampler(ds_tr.iso_code,
                                                                shuffle=BATCH_SHUFFLE))
        dl_te = DataLoader(ds_te, batch_sampler=IsoBatchSampler(ds_te.iso_code,
                                                                shuffle=False))

        model = MLP(len(FEATURES), N_HIDDEN, N_LAYERS, DROPOUT).to(DEVICE)
        opt   = torch.optim.Adam(model.parameters(), lr=LR)

        best, patience = float("inf"), 0
        for _ in range(MAX_EPOCHS):
            loss_tr = run_epoch(model, dl_tr, opt, LAM)
            if loss_tr < best - 1e-6:
                best, patience = loss_tr, 0
            else:
                patience += 1
                if patience >= PATIENCE:
                    break

        # predict
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in dl_te:
                batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                         for k,v in batch.items()}
                preds.append(model(batch["x"]).cpu())
        preds_runs.append(torch.cat(preds).numpy())

    preds_arr  = np.stack(preds_runs, axis=1)
    pred_mean  = preds_arr.mean(axis=1)

    # save CSV ----------------------------------------------------------
    df_out = pd.DataFrame({"ERG": df_test["ERG"], "XS_true": df_test["XS"]})
    for r in range(N_RUNS):
        df_out[f"pred_run{r+1}"] = preds_arr[:, r]
    df_out["pred_mean"] = pred_mean
    csv_path = out_dir / f"{iso_hold}_predictions.csv"
    df_out.to_csv(csv_path, index=False)

    # plot --------------------------------------------------------------
    plt.figure(figsize=(6,4))
    plt.plot(df_out["ERG"], df_out["XS_true"],  lw=2, label="True XS")
    plt.plot(df_out["ERG"], df_out["pred_mean"], lw=2, ls="--",
             label=f"Mean pred ({N_RUNS}×)")
    plt.xlabel("ERG"); plt.ylabel("XS")
    plt.title(f"{iso_hold} – true vs predicted XS")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / f"{iso_hold}_erg_vs_pred.png", dpi=300)
    plt.close()

    # meta --------------------------------------------------------------
    meta = dict(lam=LAM, n_hidden=N_HIDDEN, n_layers=N_LAYERS,
                dropout=DROPOUT, lr=LR, n_runs=N_RUNS,
                iso_held_out=iso_hold)
    with open(out_dir / "run_meta.json", "w") as fh:
        json.dump(meta, fh, indent=2)

    print(f"✓ finished {iso_hold}")

# ------------------------------- MAIN ---------------------------------
def main() -> None:
    BASE_OUT.mkdir(parents=True, exist_ok=True)
    df_all   = pd.read_csv(CSV_PATH)
    iso_list = sorted(df_all[ISO].unique())

    print(f"Found {len(iso_list)} unique isotopes.")
    for iso in iso_list:
        process_isotope(df_all, iso)

    print("\nAll isotopes done!")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()

In [ ]:
# --------------------------- IMPORTS ----------------------------------
import random, pathlib, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --------------------------- CONSTANTS --------------------------------
CSV_PATH   = "merged_8_july_d_lowmassremoved.csv"          # master dataset
BASE_OUT   = pathlib.Path("results_lam0p8")   # parent folder for all isotopes
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"


LAM        = 0.8
N_HIDDEN   = 192
N_LAYERS   = 4
DROPOUT    = 0.13593330178635776
LR         = 0.0007731610563105644

MAX_EPOCHS = 200
PATIENCE   = 15
BATCH_SHUFFLE = True

N_RUNS     = 10
SEED_BASE  = 1234
EPS        = 1e-12

FEATURES = [
    'ERG','Radius','n_rms_radius','p_rms_radius','octopole_deformation',
    'n_chem_erg','Sn','Z','S2p_compound','gamma_deformation','ME',
    'BEA_A_daughter','A','Radius_daughter','BEA_A','AM','Radius_compound',
    'Pairing_daughter','BEA_compound','beta_deformation','Theory_thresh',
    'shell_P','pairing_delta','symmetry_S','excitation_energy',
    'Thresh_n3n','Excite_n3n','Thresh_n4n','Excite_n4n'
]
TARGET, SIGMA_EXP, FLUENCE, GAMMA, ISO = (
    "XS","sigma_exp","fluence","gamma","iso_id"
)

# --------------------------- UTILITIES --------------------------------
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ----------------------- DATASET & SAMPLER ----------------------------
class XSData(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler,
                 fit_scaler: bool = False):
        if fit_scaler:
            scaler.fit(df[FEATURES])
        self.X = torch.tensor(scaler.transform(df[FEATURES]).astype(np.float32))
        self.y = torch.tensor(df[TARGET].to_numpy(np.float32))
        self.fluence   = torch.tensor(df[FLUENCE].to_numpy(np.float32))
        self.sigma_exp = torch.tensor(df[SIGMA_EXP].fillna(0).to_numpy(np.float32))
        self.gamma     = torch.tensor(df[GAMMA].to_numpy(np.float32))
        self.iso_code  = df[ISO].astype("category").cat.codes.to_numpy()

    def __len__(self): return len(self.y)

    def __getitem__(self, idx: int):
        return {"x": self.X[idx], "y": self.y[idx],
                "fluence": self.fluence[idx],
                "sigma_exp": self.sigma_exp[idx],
                "gamma": self.gamma[idx],
                "iso_code": self.iso_code[idx]}

class IsoBatchSampler(Sampler):
    def __init__(self, iso_codes, shuffle=True):
        self.indices = {}
        for i, iso in enumerate(iso_codes):
            self.indices.setdefault(int(iso), []).append(i)
        self.keys = list(self.indices.keys())
        self.shuffle = shuffle

    def __iter__(self):
        keys = self.keys.copy()
        if self.shuffle: random.shuffle(keys)
        for k in keys:
            yield self.indices[k]

    def __len__(self): return len(self.keys)

# ------------------------------ MODEL ---------------------------------
class MLP(nn.Module):
    def __init__(self, n_in:int, n_hidden:int, n_layers:int, p_drop:float):
        super().__init__()
        layers = []
        for l in range(n_layers):
            layers += [nn.Linear(n_in if l==0 else n_hidden, n_hidden),
                       nn.ReLU(), nn.Dropout(p_drop)]
        layers.append(nn.Linear(n_hidden, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x).squeeze(1)

# -------------------------- CUSTOM LOSS -------------------------------
def iso_loss(batch, preds, lam:float):
    mse = nn.functional.mse_loss(preds, batch["y"])
    if batch["gamma"][0] == 0:     # skip σ₁g term if γ = 0
        return mse
    flu        = batch["fluence"]
    sigma_pred = (flu * preds).sum() / (flu.sum() + EPS)
    sigma_true = batch["sigma_exp"][0]
    rel_err_sq = ((sigma_pred - sigma_true) / (sigma_true + EPS))**2
    return mse + lam * rel_err_sq

# ------------------------ TRAIN / VALID LOOP --------------------------
def run_epoch(model, loader, opt, lam):
    is_train = opt is not None
    model.train(is_train)
    total, n = 0.0, 0
    for batch in loader:
        batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                 for k,v in batch.items()}
        preds = model(batch["x"])
        loss  = iso_loss(batch, preds, lam)
        if is_train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        total += loss.item() * len(preds)
        n     += len(preds)
    return total / n

# ----------------------- PER-ISOTOPE ROUTINE --------------------------
def process_isotope(df_all: pd.DataFrame, iso_hold: str) -> None:
    out_dir = BASE_OUT / iso_hold
    out_dir.mkdir(parents=True, exist_ok=True)

    df_test  = df_all[df_all[ISO] == iso_hold].reset_index(drop=True)
    df_train = df_all[df_all[ISO] != iso_hold].reset_index(drop=True)
    if df_test.empty:
        print(f"⚠  No rows for iso_id={iso_hold}; skipping.")
        return

    preds_runs = []
    for run in range(N_RUNS):
        set_seed(SEED_BASE + run)

        scaler = StandardScaler()
        ds_tr  = XSData(df_train, scaler, fit_scaler=True)
        ds_te  = XSData(df_test,  scaler, fit_scaler=False)

        dl_tr = DataLoader(ds_tr, batch_sampler=IsoBatchSampler(ds_tr.iso_code,
                                                                shuffle=BATCH_SHUFFLE))
        dl_te = DataLoader(ds_te, batch_sampler=IsoBatchSampler(ds_te.iso_code,
                                                                shuffle=False))

        model = MLP(len(FEATURES), N_HIDDEN, N_LAYERS, DROPOUT).to(DEVICE)
        opt   = torch.optim.Adam(model.parameters(), lr=LR)

        best, patience = float("inf"), 0
        for _ in range(MAX_EPOCHS):
            loss_tr = run_epoch(model, dl_tr, opt, LAM)
            if loss_tr < best - 1e-6:
                best, patience = loss_tr, 0
            else:
                patience += 1
                if patience >= PATIENCE:
                    break

        # predict
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in dl_te:
                batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                         for k,v in batch.items()}
                preds.append(model(batch["x"]).cpu())
        preds_runs.append(torch.cat(preds).numpy())

    preds_arr  = np.stack(preds_runs, axis=1)
    pred_mean  = preds_arr.mean(axis=1)

    # save CSV ----------------------------------------------------------
    df_out = pd.DataFrame({"ERG": df_test["ERG"], "XS_true": df_test["XS"]})
    for r in range(N_RUNS):
        df_out[f"pred_run{r+1}"] = preds_arr[:, r]
    df_out["pred_mean"] = pred_mean
    csv_path = out_dir / f"{iso_hold}_predictions.csv"
    df_out.to_csv(csv_path, index=False)

    # plot --------------------------------------------------------------
    plt.figure(figsize=(6,4))
    plt.plot(df_out["ERG"], df_out["XS_true"],  lw=2, label="True XS")
    plt.plot(df_out["ERG"], df_out["pred_mean"], lw=2, ls="--",
             label=f"Mean pred ({N_RUNS}×)")
    plt.xlabel("ERG"); plt.ylabel("XS")
    plt.title(f"{iso_hold} – true vs predicted XS")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / f"{iso_hold}_erg_vs_pred.png", dpi=300)
    plt.close()

    # meta --------------------------------------------------------------
    meta = dict(lam=LAM, n_hidden=N_HIDDEN, n_layers=N_LAYERS,
                dropout=DROPOUT, lr=LR, n_runs=N_RUNS,
                iso_held_out=iso_hold)
    with open(out_dir / "run_meta.json", "w") as fh:
        json.dump(meta, fh, indent=2)

    print(f"✓ finished {iso_hold}")

# ------------------------------- MAIN ---------------------------------
def main() -> None:
    BASE_OUT.mkdir(parents=True, exist_ok=True)
    df_all   = pd.read_csv(CSV_PATH)
    iso_list = sorted(df_all[ISO].unique())

    print(f"Found {len(iso_list)} unique isotopes.")
    for iso in iso_list:
        process_isotope(df_all, iso)

    print("\nAll isotopes done!")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()

In [ ]:
# --------------------------- IMPORTS ----------------------------------
import random, pathlib, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --------------------------- CONSTANTS --------------------------------
CSV_PATH   = "merged_8_july_d_lowmassremoved.csv"          # master dataset
BASE_OUT   = pathlib.Path("results_lam0p9")   # parent folder for all isotopes
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"


LAM        = 0.9
N_HIDDEN   = 128
N_LAYERS   = 4
DROPOUT    = 0.2149288100502335
LR         = 0.00022844523281162816

MAX_EPOCHS = 200
PATIENCE   = 15
BATCH_SHUFFLE = True

N_RUNS     = 10
SEED_BASE  = 1234
EPS        = 1e-12

FEATURES = [
    'ERG','Radius','n_rms_radius','p_rms_radius','octopole_deformation',
    'n_chem_erg','Sn','Z','S2p_compound','gamma_deformation','ME',
    'BEA_A_daughter','A','Radius_daughter','BEA_A','AM','Radius_compound',
    'Pairing_daughter','BEA_compound','beta_deformation','Theory_thresh',
    'shell_P','pairing_delta','symmetry_S','excitation_energy',
    'Thresh_n3n','Excite_n3n','Thresh_n4n','Excite_n4n'
]
TARGET, SIGMA_EXP, FLUENCE, GAMMA, ISO = (
    "XS","sigma_exp","fluence","gamma","iso_id"
)

# --------------------------- UTILITIES --------------------------------
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ----------------------- DATASET & SAMPLER ----------------------------
class XSData(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler,
                 fit_scaler: bool = False):
        if fit_scaler:
            scaler.fit(df[FEATURES])
        self.X = torch.tensor(scaler.transform(df[FEATURES]).astype(np.float32))
        self.y = torch.tensor(df[TARGET].to_numpy(np.float32))
        self.fluence   = torch.tensor(df[FLUENCE].to_numpy(np.float32))
        self.sigma_exp = torch.tensor(df[SIGMA_EXP].fillna(0).to_numpy(np.float32))
        self.gamma     = torch.tensor(df[GAMMA].to_numpy(np.float32))
        self.iso_code  = df[ISO].astype("category").cat.codes.to_numpy()

    def __len__(self): return len(self.y)

    def __getitem__(self, idx: int):
        return {"x": self.X[idx], "y": self.y[idx],
                "fluence": self.fluence[idx],
                "sigma_exp": self.sigma_exp[idx],
                "gamma": self.gamma[idx],
                "iso_code": self.iso_code[idx]}

class IsoBatchSampler(Sampler):
    def __init__(self, iso_codes, shuffle=True):
        self.indices = {}
        for i, iso in enumerate(iso_codes):
            self.indices.setdefault(int(iso), []).append(i)
        self.keys = list(self.indices.keys())
        self.shuffle = shuffle

    def __iter__(self):
        keys = self.keys.copy()
        if self.shuffle: random.shuffle(keys)
        for k in keys:
            yield self.indices[k]

    def __len__(self): return len(self.keys)

# ------------------------------ MODEL ---------------------------------
class MLP(nn.Module):
    def __init__(self, n_in:int, n_hidden:int, n_layers:int, p_drop:float):
        super().__init__()
        layers = []
        for l in range(n_layers):
            layers += [nn.Linear(n_in if l==0 else n_hidden, n_hidden),
                       nn.ReLU(), nn.Dropout(p_drop)]
        layers.append(nn.Linear(n_hidden, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x).squeeze(1)

# -------------------------- CUSTOM LOSS -------------------------------
def iso_loss(batch, preds, lam:float):
    mse = nn.functional.mse_loss(preds, batch["y"])
    if batch["gamma"][0] == 0:     # skip σ₁g term if γ = 0
        return mse
    flu        = batch["fluence"]
    sigma_pred = (flu * preds).sum() / (flu.sum() + EPS)
    sigma_true = batch["sigma_exp"][0]
    rel_err_sq = ((sigma_pred - sigma_true) / (sigma_true + EPS))**2
    return mse + lam * rel_err_sq

# ------------------------ TRAIN / VALID LOOP --------------------------
def run_epoch(model, loader, opt, lam):
    is_train = opt is not None
    model.train(is_train)
    total, n = 0.0, 0
    for batch in loader:
        batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                 for k,v in batch.items()}
        preds = model(batch["x"])
        loss  = iso_loss(batch, preds, lam)
        if is_train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        total += loss.item() * len(preds)
        n     += len(preds)
    return total / n

# ----------------------- PER-ISOTOPE ROUTINE --------------------------
def process_isotope(df_all: pd.DataFrame, iso_hold: str) -> None:
    out_dir = BASE_OUT / iso_hold
    out_dir.mkdir(parents=True, exist_ok=True)

    df_test  = df_all[df_all[ISO] == iso_hold].reset_index(drop=True)
    df_train = df_all[df_all[ISO] != iso_hold].reset_index(drop=True)
    if df_test.empty:
        print(f"⚠  No rows for iso_id={iso_hold}; skipping.")
        return

    preds_runs = []
    for run in range(N_RUNS):
        set_seed(SEED_BASE + run)

        scaler = StandardScaler()
        ds_tr  = XSData(df_train, scaler, fit_scaler=True)
        ds_te  = XSData(df_test,  scaler, fit_scaler=False)

        dl_tr = DataLoader(ds_tr, batch_sampler=IsoBatchSampler(ds_tr.iso_code,
                                                                shuffle=BATCH_SHUFFLE))
        dl_te = DataLoader(ds_te, batch_sampler=IsoBatchSampler(ds_te.iso_code,
                                                                shuffle=False))

        model = MLP(len(FEATURES), N_HIDDEN, N_LAYERS, DROPOUT).to(DEVICE)
        opt   = torch.optim.Adam(model.parameters(), lr=LR)

        best, patience = float("inf"), 0
        for _ in range(MAX_EPOCHS):
            loss_tr = run_epoch(model, dl_tr, opt, LAM)
            if loss_tr < best - 1e-6:
                best, patience = loss_tr, 0
            else:
                patience += 1
                if patience >= PATIENCE:
                    break

        # predict
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in dl_te:
                batch = {k:(v.to(DEVICE) if torch.is_tensor(v) else v)
                         for k,v in batch.items()}
                preds.append(model(batch["x"]).cpu())
        preds_runs.append(torch.cat(preds).numpy())

    preds_arr  = np.stack(preds_runs, axis=1)
    pred_mean  = preds_arr.mean(axis=1)

    # save CSV ----------------------------------------------------------
    df_out = pd.DataFrame({"ERG": df_test["ERG"], "XS_true": df_test["XS"]})
    for r in range(N_RUNS):
        df_out[f"pred_run{r+1}"] = preds_arr[:, r]
    df_out["pred_mean"] = pred_mean
    csv_path = out_dir / f"{iso_hold}_predictions.csv"
    df_out.to_csv(csv_path, index=False)

    # plot --------------------------------------------------------------
    plt.figure(figsize=(6,4))
    plt.plot(df_out["ERG"], df_out["XS_true"],  lw=2, label="True XS")
    plt.plot(df_out["ERG"], df_out["pred_mean"], lw=2, ls="--",
             label=f"Mean pred ({N_RUNS}×)")
    plt.xlabel("ERG"); plt.ylabel("XS")
    plt.title(f"{iso_hold} – true vs predicted XS")
    plt.legend(); plt.tight_layout()
    plt.savefig(out_dir / f"{iso_hold}_erg_vs_pred.png", dpi=300)
    plt.close()

    # meta --------------------------------------------------------------
    meta = dict(lam=LAM, n_hidden=N_HIDDEN, n_layers=N_LAYERS,
                dropout=DROPOUT, lr=LR, n_runs=N_RUNS,
                iso_held_out=iso_hold)
    with open(out_dir / "run_meta.json", "w") as fh:
        json.dump(meta, fh, indent=2)

    print(f"✓ finished {iso_hold}")

# ------------------------------- MAIN ---------------------------------
def main() -> None:
    BASE_OUT.mkdir(parents=True, exist_ok=True)
    df_all   = pd.read_csv(CSV_PATH)
    iso_list = sorted(df_all[ISO].unique())

    print(f"Found {len(iso_list)} unique isotopes.")
    for iso in iso_list:
        process_isotope(df_all, iso)

    print("\nAll isotopes done!")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()